In [39]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete, MultiBinary, Tuple, Dict

# Import helpers
import numpy as np
import random
import os

# Import stable-baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


# Space types

In [23]:
action = Discrete(3).sample()
print(action)

2


In [24]:
action = Box(low=0, high=1, shape=(3,)).sample()
print(action)

[0.00299133 0.31598967 0.74474305]


In [25]:
# Not supported by stable-baselines
action = Tuple((Discrete(3), Box(low=0, high=1, shape=(3,)))).sample()
print(action)

(1, array([0.3595346, 0.3664725, 0.917722 ], dtype=float32))


In [26]:
# Not supported by stable-baselines
action = Dict({"action1": Discrete(3), "action2": Box(low=0, high=1, shape=(3,))}).sample()
print(action)

OrderedDict([('action1', 0), ('action2', array([0.44328335, 0.69975626, 0.44148055], dtype=float32))])


In [27]:
action = MultiDiscrete([3, 2, 2]).sample()
print(action)

[0 1 1]


In [28]:
action = MultiBinary(3).sample()
print(action)

[0 0 0]


# Building environment


In [44]:
import gym
import numpy as np
from gym import spaces


class ShowerEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super().__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(3)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60

    def step(self, action):
        self.state += action - 1
        self.shower_length -= 1
        
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info

    def render():
        pass

    def reset(self):
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state

    def close(self):
        pass

In [45]:
env = ShowerEnv()
# print(env.action_space.sample())

In [43]:
print(env.action_space)

<class 'gym.spaces.discrete.Discrete'>


In [48]:
# Train model
log_path = os.path.join("Training", "Logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [49]:
model.learn(total_timesteps=10000)

Logging to Training\Logs\PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -32.7    |
| time/              |          |
|    fps             | 1451     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -28.1        |
| time/                   |              |
|    fps                  | 1011         |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0077619976 |
|    clip_fraction        | 0.0449       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   |

In [50]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward}")

c:\Users\duppu\miniconda3\envs\introtoai\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:0.00 +/- 60.0
